# Dados cartográficos com Python

## GeoPandas

GeoPandas é uma biblioteca do Python que estende as funcionalidades do Pandas, permitindo que se trabalhe de maneira eficiente com dados geoespaciais. Ela é construída sobre outras bibliotecas populares como Shapely, Fiona e Pyproj, o que lhe permite manipular, analisar e visualizar dados geoespaciais de forma integrada e simplificada.

A principal característica do GeoPandas é a capacidade de lidar com geometrias (pontos, linhas, polígonos, etc.) em colunas especiais chamadas de `GeoSeries`. Essas geometrias podem ser usadas para representar entidades espaciais, como localizações geográficas, fronteiras de cidades, ou trajetórias. Cada linha de um `GeoDataFrame`—a versão geoespacial do `DataFrame` no Pandas—pode conter uma geometria associada a outros atributos, permitindo que se façam análises espaciais sobre os dados.

Com o GeoPandas, tarefas comuns em análise geoespacial, como a leitura de arquivos shapefile, a re-projeção de sistemas de coordenadas e operações espaciais (interseção, união, diferenciação) tornam-se muito mais fáceis. A biblioteca é amplamente usada em aplicações que envolvem geoprocessamento, cartografia e estudos de geografia.

GeoPandas também se integra bem com bibliotecas de visualização como Matplotlib, facilitando a criação de mapas e gráficos baseados em dados espaciais. Ela é amplamente usada em estudos ambientais, urbanos, logísticos e em qualquer área que necessite de análise espacial eficiente.

https://geopandas.org/en/stable/index.html

## Folium

Folium é uma biblioteca do Python voltada para a criação de mapas interativos utilizando Leaflet.js, uma popular biblioteca JavaScript de mapeamento. O Folium permite que se criem mapas ricos e dinâmicos diretamente em Python, integrando dados geoespaciais de maneira simples e intuitiva.

Uma das principais vantagens do Folium é a facilidade com que ele transforma dados geoespaciais, como pontos de interesse, trajetos ou áreas geográficas, em visualizações interativas. Ele suporta uma ampla gama de camadas, como marcadores, polígonos, retângulos e círculos, que podem ser facilmente sobrepostos em diferentes tipos de mapas base, como mapas de ruas ou satélites. Além disso, a biblioteca permite adicionar informações complementares por meio de popups e tooltips, o que facilita a apresentação de dados complexos de forma clara e acessível.

Folium é frequentemente utilizado em projetos que requerem visualizações de dados espaciais interativas, como análise de trajetórias, planejamento urbano, monitoramento ambiental e estudos logísticos. Sua integração com outras bibliotecas como Pandas e GeoPandas permite manipular dados e criar mapas de forma eficiente. A biblioteca também facilita a exportação dos mapas para HTML, o que é útil para incorporá-los em sites ou relatórios interativos.

De maneira geral, Folium oferece uma solução para transformar dados geoespaciais em mapas interativos, sendo uma escolha popular tanto em análises exploratórias quanto na apresentação final de resultados.

https://python-visualization.github.io/folium/latest/#

In [9]:
import folium
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from folium import plugins
from shapely.geometry import Point

from src.config import DADOS_LIMPOS, DADOS_GEO_ORIGINAIS, DADOS_GEO_MEDIAN
from src.graficos import SCATTER_ALPHA, PALETTE
import warnings

# Ignora todos os warnings
warnings.filterwarnings("ignore")

sns.set_theme(style="white", palette="bright")

In [10]:
df = pd.read_parquet(DADOS_LIMPOS)

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,median_income_cat,rooms_per_household,population_per_household,bedrooms_per_room
0,-122.230003,37.880001,41,880,129,322,126,8.3252,452600,NEAR BAY,5,6.984127,2.555556,0.146591
1,-122.220001,37.860001,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY,5,6.238137,2.109842,0.155797
2,-122.260002,37.840000,42,2555,665,1206,595,2.0804,226700,NEAR BAY,2,4.294117,2.026891,0.260274
3,-122.260002,37.849998,50,1120,283,697,264,2.1250,140000,NEAR BAY,2,4.242424,2.640152,0.252679
4,-122.260002,37.840000,50,2239,455,990,419,1.9911,158700,NEAR BAY,2,5.343676,2.362768,0.203216


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17564 entries, 0 to 17563
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   longitude                 17564 non-null  float32 
 1   latitude                  17564 non-null  float32 
 2   housing_median_age        17564 non-null  int8    
 3   total_rooms               17564 non-null  int16   
 4   total_bedrooms            17564 non-null  int16   
 5   population                17564 non-null  int16   
 6   households                17564 non-null  int16   
 7   median_income             17564 non-null  float32 
 8   median_house_value        17564 non-null  int32   
 9   ocean_proximity           17564 non-null  category
 10  median_income_cat         17564 non-null  int64   
 11  rooms_per_household       17564 non-null  float32 
 12  population_per_household  17564 non-null  float32 
 13  bedrooms_per_room         17564 non-null  floa

O melhor termo para traduzir **"county"** seria **"condado"**. Embora o Brasil não tenha uma divisão administrativa exata equivalente aos condados dos Estados Unidos, "condado" é uma tradução apropriada e amplamente compreendida.

Nos EUA, os **counties** (condados) são subdivisões dos estados, com certa autonomia administrativa, similar às regiões intermediárias entre estados e municípios no Brasil. No entanto, a melhor correspondência prática no Brasil, considerando a divisão político-administrativa, seria com os **municípios**, que são as subdivisões administrativas dos estados brasileiros, apesar de suas diferenças em termos de autonomia e funções.


In [13]:
gdf_counties = gpd.read_file(DADOS_GEO_ORIGINAIS)

gdf_counties.head()

DataSourceError: 'C:\Users\useca\Desktop\Julio Python\JV Regressão com Scikit-Learn - Algoritmos Lineares\Projeto de Regressão\modelo_projeto_data_science\dados\california_counties.geojson' not recognized as being in a supported file format. It might help to specify the correct driver explicitly by prefixing the file path with '<DRIVER>:', e.g. 'CSV:path'.

In [ ]:
pontos = [Point(long, lat) for long, lat in zip(df["longitude"], df["latitude"])]

pontos[:5]

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry= pontos)

gdf.head()

In [ ]:
gdf.info()

Em um arquivo GeoJSON, colunas como **"abcode"** e **"ansi"** contêm códigos padronizados que identificam de maneira única regiões geográficas, divisões administrativas ou entidades espaciais.

A coluna **"abcode"** representa **códigos de fronteira administrativa** (Administrative Boundary Code). Esses códigos são usados para identificar divisões administrativas, como estados, condados, cidades ou outras subdivisões de um país. Dependendo do contexto, "abcode" pode estar relacionado a códigos nacionais ou internacionais de áreas administrativas.

Por exemplo:
- Nos Estados Unidos, o "abcode" pode se referir a códigos FIPS (Federal Information Processing Standards), usados para identificar estados e condados.
- Em outros países, pode ser um código específico para uma província, estado ou outra divisão territorial.

A coluna **"ansi"** refere-se aos códigos do **American National Standards Institute (ANSI)** (American National Standards Institute Code), usados nos Estados Unidos para identificar regiões geográficas. O ANSI substituiu os códigos FIPS em 2008 como padrão de identificação de entidades geográficas, como estados e condados.

Por exemplo:
- Nos EUA, cada estado e condado tem um **ANSI code** único que os identifica de forma oficial. Para estados, o ANSI code é o mesmo que o código FIPS (geralmente um número de 2 dígitos), e para condados, é um número de 5 dígitos.

Assim: 

- **abcode**: Refere-se a um código de fronteira administrativa, usado para identificar divisões geográficas ou administrativas específicas, como estados, cidades ou condados.
- **ansi**: Refere-se a códigos padronizados pelo ANSI, usados principalmente nos EUA para identificar estados e condados. Esses códigos são uma forma de identificar regiões geográficas de maneira única e padronizada.

In [ ]:
gdf = gdf.set_crs(epsg = 4326)

gdf_counties = gdf_counties.to_crs(epsg = 4326)

In [ ]:
gdf.crs

In [ ]:
gdf_counties.crs

### Sobre sistemas de referência

Um Sistema de Referência de Coordenadas (CRS, do inglês *Coordinate Reference System*) é o conjunto de regras que define como os dados geoespaciais são projetados na superfície da Terra. Imagine o CRS como uma forma de "tradução" que nos ajuda a entender e posicionar os dados geográficos corretamente em um mapa. Cada ponto no mapa (como uma cidade, uma montanha ou uma estrada) tem coordenadas que dependem do CRS usado. Essas coordenadas podem ser expressas em diferentes sistemas, como latitudes e longitudes ou distâncias em metros.

Por que isso é importante? Quando trabalhamos com dados geoespaciais, especialmente em formatos como GeoJSON, Shapefiles ou outros, o CRS nos diz como interpretar as coordenadas. Por exemplo, o sistema mais comum que você verá é o WGS 84 (EPSG:4326), que usa latitude e longitude para posicionar qualquer ponto na Terra. No entanto, em projetos locais ou regionais, outros sistemas de referência podem ser usados, que distorcem menos as áreas específicas.

Quando você abre um arquivo GeoJSON ou outro tipo de dado geoespacial, muitas vezes verá algo como "crs": {"type": "name", "properties": {"name": "EPSG:4326"}}. Isso está simplesmente informando qual sistema está sendo usado para mapear as coordenadas no arquivo. Se diferentes arquivos estiverem em CRSs distintos, será necessário convertê-los para o mesmo sistema, ou eles não se alinharão corretamente quando visualizados juntos.



**WGS 84** é um sistema geodésico global que define um modelo matemático para a forma da Terra. Ele inclui um elipsoide de referência (uma representação matemática da Terra), um sistema de coordenadas cartesianas (X, Y, Z) e um sistema de coordenadas geográficas (latitude, longitude e altura). Esse sistema é amplamente usado em GPS e outros sistemas de posicionamento global, sendo uma das referências mais comuns para medir e mapear a superfície da Terra.

**EPSG:4326**, por outro lado, é o código de identificação que faz parte de uma base de dados chamada **EPSG Registry**, que contém descrições de vários sistemas de referência de coordenadas usados globalmente. O código **4326** se refere especificamente ao CRS que utiliza o **WGS 84** como sistema geodésico de base e expressa coordenadas em graus de latitude e longitude. Esse código é amplamente utilizado em formatos geoespaciais, como GeoJSON, para identificar que o sistema de coordenadas é o WGS 84.

Então, podemos dizer que **WGS 84** é o sistema geodésico, e **EPSG:4326** é o código que referencia esse sistema específico dentro de um conjunto de padrões geoespaciais. Eles estão intimamente relacionados, pois EPSG:4326 usa WGS 84, mas o EPSG:4326 é apenas uma forma de identificar esse sistema em softwares e arquivos.

#### Relação com as projeções de mapas

A Terra tem uma forma quase esférica (na verdade, ela é um esferoide oblato, ligeiramente achatada nos polos), mas os mapas são geralmente representações bidimensionais. Isso gera um desafio: como transformar a superfície curva da Terra em um mapa plano? É aí que entram as **projeções cartográficas**.

**Projeção cartográfica** é o método utilizado para "achatar" a superfície da Terra em um plano, permitindo a criação de mapas. Cada projeção tenta preservar algumas características da Terra (como áreas, formas ou distâncias), mas sempre há algum tipo de distorção, já que é impossível representar uma superfície curva em um plano sem perdas.

Agora, voltando ao **WGS 84** e **EPSG:4326**:

- **WGS 84** é um sistema de referência global que define coordenadas em latitude e longitude, como mencionado antes, sem aplicar diretamente uma projeção cartográfica. Ou seja, ele usa um sistema geodésico para definir onde os pontos estão na superfície curva da Terra, sem "achatar" essa superfície em um mapa.

- Quando usamos **EPSG:4326**, estamos trabalhando com coordenadas de latitude e longitude, mas sem projetar essas coordenadas em uma forma plana (ainda estamos no sistema esférico). Essa é uma das formas mais comuns de representar dados geoespaciais porque é universal e fácil de entender, mas não é exatamente uma projeção cartográfica.

As projeções cartográficas que aprendemos na escola—como a **projeção de Mercator** (que distorce áreas perto dos polos), a **projeção de Peters** (que tenta preservar áreas), ou a **projeção azimutal** (que preserva direções a partir de um ponto central)—são formas de transformar as coordenadas da Terra em um plano. Dependendo da finalidade do mapa, uma projeção pode ser mais adequada que outra.

Por exemplo:

- A **projeção de Mercator** é útil para navegação porque preserva ângulos e direções, mas distorce o tamanho das regiões conforme se aproximam dos polos.
- A **projeção de Peters** tenta representar as áreas de forma mais precisa, mas distorce as formas dos continentes.

Quando usamos um CRS como o **EPSG:4326** (WGS 84), estamos basicamente posicionando pontos na superfície curva da Terra. Para "achatar" esses pontos em um mapa, muitas vezes usamos uma projeção cartográfica. Por exemplo, se quisermos criar um mapa que use uma projeção de Mercator, utilizamos outro CRS específico para essa projeção, como o **EPSG:3857**, que é a projeção de Mercator.

Em resumo, os conceitos de sistemas de referência de coordenadas (CRS) e projeções cartográficas estão conectados. O CRS nos dá as coordenadas na superfície curva da Terra, e a projeção cartográfica define como essas coordenadas serão representadas em um mapa plano.

https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins

https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html

In [ ]:
gdf_joined = gpd.sjoin(gdf, gdf_counties, how = "left", predicate = "within")

gdf_joined

In [ ]:
gdf_joined = gdf_joined.drop(columns= ["index_right", "fullname","abcode", "ansi"])

gdf_joined.head()

In [ ]:
gdf_joined.info()

In [ ]:
gdf_joined.isnull().sum()

In [ ]:
gdf_joined.loc[gdf_joined["name"].isnull() & gdf_joined["abbrev"].isnull()]

In [ ]:
linhas_faltantes  = gdf_joined.loc[gdf_joined["name"].isnull() & gdf_joined["abbrev"].isnull()].index


linhas_faltantes

O **centroide** em dados geográficos é o ponto que representa o centro geométrico de uma forma ou área. Em termos simples, é o ponto médio de uma geometria, como um polígono ou multipolígono, que pode representar uma cidade, um estado ou qualquer outra divisão geográfica.

No contexto do **GeoPandas**, o centroide é uma propriedade útil para resumir a localização de uma área. Por exemplo, se você tem o contorno de um município ou de uma região, o centroide é o ponto que melhor representa o "centro" dessa área, mesmo que sua forma seja irregular. O centroide é calculado considerando as coordenadas da geometria.

No **GeoPandas**, você pode calcular o centroide de uma geometria utilizando o atributo `.centroid`. Isso gera um ponto, que pode ser usado para várias finalidades, como identificar o ponto central de uma área no mapa, fazer comparações de distâncias entre diferentes regiões, ou usá-lo em visualizações.

Por exemplo, se você tem um conjunto de regiões geográficas e deseja identificar seus pontos centrais para colocar marcadores em um mapa, o centroide é ideal para essa finalidade.

- Se você tem o contorno de um país ou estado, o centroide será o ponto mais central dessa área.
- Mesmo para áreas irregulares, como fronteiras naturais ou regiões com formatos complexos, o centroide ainda é o ponto médio calculado geometricamente.


In [ ]:
gdf_counties["centroid"] = gdf_counties.centroid

gdf_counties.head()

In [ ]:
print(gdf_joined.loc[1507, "geometry"])

In [ ]:
gdf_counties["centroid"].distance(gdf_joined.loc[1507, "geometry"])

In [ ]:
gdf_counties["centroid"].distance(gdf_joined.loc[1507, "geometry"]).idxmin()

In [ ]:
gdf_counties["centroid"].distance(gdf_joined.loc[1507, "geometry"]).min()

In [ ]:
gdf_counties.iloc[1]

In [ ]:
def condado_mais_proximo(linha):
    ponto = linha["geometry"]
    distancia = gdf_counties["centroid"].distance(ponto)
    idx_condado_mais_proximo = distancia.idxmin()
    condado_mais_proximo = gdf_counties.loc[idx_condado_mais_proximo]
    return condado_mais_proximo[["name", "abbrev"]]

In [ ]:
condado_mais_proximo(gdf_joined.loc[1507])

In [ ]:
gdf_joined.loc[gdf_joined["name"].isnull() & gdf_joined["abbrev"].isnull()].index


In [ ]:
gdf_joined.loc[linhas_faltantes, ["name", "abbrev"]] = gdf_joined.loc[linhas_faltantes].apply(condado_mais_proximo, axis =1)

In [ ]:
gdf_joined.isnull().sum()

In [ ]:
gdf_joined["name"].value_counts()

In [ ]:
gdf_counties.plot()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))

gdf_counties.plot(
    ax = ax,
    edgecolor = "black"
)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))

gdf_counties.plot(
    ax = ax,
    edgecolor = "black",
    color = "lightgray"
)

ax.scatter(
    gdf_joined["longitude"],
    gdf_joined["latitude"],
    color = "red",
    s = 1,
    alpha = SCATTER_ALPHA
)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))

gdf_counties.plot(
    ax = ax,
    edgecolor = "black",
    color = "lightgray"
)

ax.scatter(
    gdf_joined["longitude"],
    gdf_joined["latitude"],
    color = "red",
    s = 1,
    alpha = SCATTER_ALPHA
)

for x,y, abbrev in zip(gdf_counties["centroid"].x, gdf_counties["centroid"].y,gdf_counties["abbrev"]):
    ax.text(x,y, abbrev, ha = "center", va = "center", fontsize = 8)

plt.show()

In [ ]:
gdf_joined.groupby("name").median(numeric_only= True).head()

In [ ]:
gdf_counties = gdf_counties.merge(
    gdf_joined.groupby("name").median(numeric_only= True),
    left_on= "name",
    right_index= True
)

In [ ]:
gdf_counties.head()

In [ ]:
gdf_counties.info()

In [ ]:
gdf_joined[["name", "ocean_proximity"]].groupby("name").describe()

In [ ]:
count_ocean_mode = gdf_joined[["name", "ocean_proximity"]].groupby("name").agg(pd.Series.mode)

In [ ]:
gdf_counties = gdf_counties.merge(
    count_ocean_mode,
    left_on= "name",
    right_index= True
)

In [ ]:
gdf_counties.head()

In [ ]:
gdf_counties.info()

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))

gdf_counties.plot(
    ax = ax,
    edgecolor = "black",
    column = "median_house_value",
    cmap = PALETTE, 
    legend = True,
    legend_kwds = {"label": "Median House Value"}
)

for x,y, abbrev in zip(gdf_counties["centroid"].x, gdf_counties["centroid"].y,gdf_counties["abbrev"]):
    ax.text(x,y, abbrev, ha = "center", va = "center", fontsize = 8)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))

gdf_counties.plot(
    ax = ax,
    edgecolor = "black",
    column = "median_income",
    cmap = PALETTE, 
    legend = True,
    legend_kwds = {"label": "Median Income Value"}
)

for x,y, abbrev in zip(gdf_counties["centroid"].x, gdf_counties["centroid"].y,gdf_counties["abbrev"]):
    ax.text(x,y, abbrev, ha = "center", va = "center", fontsize = 8)

plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20,7))

gdf_counties.plot(
    ax = axs[0],
    edgecolor = "black",
    column = "median_house_value",
    cmap = PALETTE, 
    legend = True,
    legend_kwds = {"label": "Median House Value"}
)
gdf_counties.plot(
    ax = axs[1],
    edgecolor = "black",
    column = "median_income",
    cmap = "YlOrRd", 
    legend = True,
    legend_kwds = {"label": "Median Income Value"}
)

for x,y, abbrev in zip(gdf_counties["centroid"].x, gdf_counties["centroid"].y,gdf_counties["abbrev"]):
    axs[0].text(x,y, abbrev, ha = "center", va = "center", fontsize = 8)
    axs[1].text(x,y, abbrev, ha = "center", va = "center", fontsize = 8)

plt.show()

In [ ]:
#gdf_counties.to_parquet(DADOS_GEO_MEDIAN)

In [57]:
centro_mapa = [df["latitude"].mean(), df["longitude"].mean()]

centro_mapa

[35.6022, -119.509995]

In [63]:
tamanho_mapa_folium = {"width": 500, "height":500}

fig = folium.Figure(**tamanho_mapa_folium)

mapa = folium.Map(location= centro_mapa).add_to(fig)
mapa

In [71]:
fig = folium.Figure(**tamanho_mapa_folium)

mapa = folium.Map(
    location= centro_mapa,
    zoom_start= 5,
    tiles = "cartodb positron"
).add_to(fig)
mapa

In [73]:
fig = folium.Figure(**tamanho_mapa_folium)

mapa = folium.Map(
    location= centro_mapa,
    zoom_start= 5,
    tiles = "cartodb voyager"
).add_to(fig)
mapa